<h3>CloudFormation</h3>

Create budget in AWS <br/>
https://docs.aws.amazon.com/awsaccountbilling/latest/aboutv2/budgets-create.html

Create S3 bucket

In [ ]:
"lambda_functions" : {
      "Type" : "AWS::S3::Bucket"
      }

Copy lambda functions to S3 bucket

Lamda function should be on github in a zip file

In [ ]:
#https://aws.amazon.com/premiumsupport/knowledge-center/start-stop-lambda-cloudwatch/

import boto3

ec2 = boto3.resource('ec2')

def lambda_handler(event, context):
    # TODO implement
    
    instances = ec2.instances.filter(
        Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])
    instanceListNames = []
    attachedVolumes = []
    for instance in instances:
        print(instance.id, instance.instance_type, instance.key_name, instance.private_ip_address, instance.public_dns_name ,instance.root_device_name)
        instanceListNames.append(instance.id)
        volumeId = instance.block_device_mappings
        attachedVolumes.append(volumeId[0]['Ebs']['VolumeId'])
        print(volumeId)

    # Check budget
    # Create a budget on AWS
    # https://docs.aws.amazon.com/awsaccountbilling/latest/aboutv2/budgets-create.html
    # 
    # Check your accoutn id
    # https://docs.aws.amazon.com/IAM/latest/UserGuide/console_account-alias.html
    # 
    
    # Get acciunt ID
    ACCOUNT_ID = context.invoked_function_arn.split(":")[4]

    client = boto3.client('budgets')
    response = client.describe_budget(
        AccountId=ACCOUNT_ID,[]
        BudgetName='BudgetInvestigation'
    )

    actualSpend = float(response['Budget']['CalculatedSpend']['ActualSpend']['Amount'])

    print(actualSpend)


    # If limit is exceeded create a snapshot of EBS and skill all instances

    if(actualSpend > 0):
        print('create snapshot')
        print(attachedVolumes)
        '''
        for volumeId in attachedVolumes:
            ec2.create_snapshot(VolumeId=volumeId)
        '''
        print('kill all instances')
        print(instanceListNames)
        '''
        ec2.instances.filter(InstanceIds=instanceListNames).stop()
        ec2.instances.filter(InstanceIds=instanceListNames).terminate()
        '''

    return 'Instances terminated'

Copy Lambda Function from S3

In [ ]:
"AMIIDLookup": {
  "Type": "AWS::Lambda::Function",
  "Properties": {
    "Handler": "aws_monitor_instances_lambda.lambda_handler",
    "Role": { "Fn::GetAtt" : ["LambdaExecutionRole", "Arn"] },
    "Code": {
      "S3Bucket": "lambda_functions",
      "S3Key": "aws_monitor_instances_lambda.zip"
    },
    "Runtime": "python3.6",
    "Timeout": "25",
    "TracingConfig": {
      "Mode": "Active"
   }
  }
}

Create an Event which call lambda function

In [ ]:
"ScheduledRule": {
  "Type": "AWS::Events::Rule",
  "Properties": {
    "Description": "checkBudget",
    "ScheduleExpression": "rate(1 minute)",
    "State": "ENABLED",
    "Targets": [{
      "Arn": { "Fn::GetAtt": ["aws_monitor_instances_lambda", "Arn"] },
      "Id": "TargetFunctionV1"
    }]
  }
},
"PermissionForEventsToInvokeLambda": {
  "Type": "AWS::Lambda::Permission",
  "Properties": {
    "FunctionName": { "Ref": "LambdaFunction" },
    "Action": "lambda:InvokeFunction",
    "Principal": "events.amazonaws.com",
    "SourceArn": { "Fn::GetAtt": ["ScheduledRule", "Arn"] }
  }
}